In [ ]:
import pandas as pd
from dateutil import parser
import datetime
import pytz

localtz = pytz.timezone('US/Eastern')

sns.set(style='white', palette='Set2')

df = pd.read_pickle('full_mlarray.pkl')

df.index = df.index.tz_convert('US/Eastern')

In [ ]:
#for pm_mlarray, cut down to hour resolution

#resample columns by hour, just to get the right structure.  This averages values over the hour.
pm_array = fullset.resample('60T',base=0).mean()

#for the particulate, we only want to use the values 0-50min on every hour.  10am captures 10:00-10:50am
first_hour = fullset.index[0].replace(minute=0, second=0, microsecond=0)
final_hour = fullset.index[-1].replace(minute=0, second=0, microsecond=0)
t_delta = datetime.timedelta(minutes=49)
t_step = datetime.timedelta(hours=1)

print first_hour
print final_hour
print first_hour + t_delta
print first_hour + t_step

print len(pm_array)



for col in fullset.columns.values:
        
        next_col = []
        current_hour = first_hour
        
        while current_hour <= final_hour:
            next_col.append(fullset.ix[current_hour:current_hour+t_delta][col].sum())
            current_hour = current_hour + t_step

        pm_array[col] = next_col
        
#currently it sums over the 50 minute period.  If we want an average, we need to divide by 50.  This will do that
#for the provided columns:
print pm_array.columns.values

#cols_to_average= []
cols_to_average = pm_array.columns.values

for col in cols_to_average:
    pm_array[col] = pm_array[col]/50.0



pm_array.tail(5)

In [ ]:
#Save Everything

as_breaktime = datetime.datetime(2016,5,23,21,40)
sck_breaktime = datetime.datetime(2016,6,6,8,52)


#particulate array
pm_array.to_pickle('pm_mlarray.pkl')
print len(pm_array)

#sck array
fullset.ix[:sck_breaktime].to_pickle('sck_mlarray.pkl')
print len(fullset.ix[:sck_breaktime])

#alphasense 1 array
fullset.ix[:as_breaktime].to_pickle('as1_mlarray.pkl')
print len(fullset.ix[:as_breaktime])

#alphasense 2 array
fullset.ix[as_breaktime:].to_pickle('as2_mlarray.pkl')
print len(fullset.ix[as_breaktime:])
